In [7]:
import datetime
import os
import warnings

import pandas as pd
import nltk
import torch

from blurr.text.data.all import *
from blurr.text.modeling.all import *
from fastai.data.block import RandomSplitter
from transformers import BartForConditionalGeneration

# silence all the HF warnings
warnings.simplefilter("ignore")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/team_007/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Grab our topics and transcripts

In [8]:
! ls ../../data/raw

fsdl_2022_project_transcripts.xlsx


In [9]:
sheets_d = pd.read_excel(
    "../../data/raw/fsdl_2022_project_transcripts.xlsx",
    sheet_name=["lesson_topics", "lesson_transcripts"],
    engine="openpyxl",
)
topics_df, transcripts_df = [v for k, v in sheets_d.items()]

topics_df.drop(columns="video_url", inplace=True)
transcripts_df.drop(columns="video_url", inplace=True)

topics_df["timestamp"] = topics_df["timestamp"].astype(str)
transcripts_df["timestamp"] = transcripts_df["timestamp"].astype(str)

In [10]:
print(len(topics_df))

topics_df.head()

600


,course_title,lesson_num,timestamp,topic
0,fast.ai 2022 - Part 1,2,00:00:00,Introduction
1,fast.ai 2022 - Part 1,2,00:00:55,Reminder to use the fastai book as a companion to the course
2,fast.ai 2022 - Part 1,2,00:02:06,aiquizzes.com for quizzes on the book
3,fast.ai 2022 - Part 1,2,00:02:36,"Reminder to use fastai forums for links, notebooks, questions, etc."
4,fast.ai 2022 - Part 1,2,00:03:42,How to efficiently read the forum with summarizations


In [11]:
print(len(transcripts_df))

transcripts_df.head()

25283


,course_title,lesson_num,timestamp,transcript
0,fast.ai 2022 - Part 1,2,00:00:00,"Hi everybody. Welcome to lesson two. Thanks for coming back… slight change of environment here,"
1,fast.ai 2022 - Part 1,2,00:00:08,we had a bit of an “administrative issue” at our university — somebody booked our room — so I'm
2,fast.ai 2022 - Part 1,2,00:00:14,doing this from the study at home. so sorry about the lack of decorations behind me.
3,fast.ai 2022 - Part 1,2,00:00:25,I'm actually really really pumped about this lesson. It feels like going back to what things
4,fast.ai 2022 - Part 1,2,00:00:32,"were like in the very early days, because we're doing some really new, really cool stuff, which…"


## Define a utility function for converting durations to total_seconds

In [12]:
def convert_duration_to_seconds(v):
    hrs, mins, secs = v.split(":")
    return (60 * 60 * int(hrs)) + (60 * int(mins)) + int(secs)

## Define the start/end boundaries (in seconds) for each topic in each lesson

In [13]:
topics_df["start_seconds"] = topics_df["timestamp"].apply(convert_duration_to_seconds)
topics_df["end_seconds"] = topics_df.groupby(by=["course_title", "lesson_num"])["start_seconds"].shift(
    -1, fill_value=100000
)

## Define the total number of elapsed seconds at each timestamp in the transcripts dataset

In [14]:
transcripts_df["elapsed_seconds"] = transcripts_df["timestamp"].apply(convert_duration_to_seconds)

In [15]:
transcripts_df.head()

,course_title,lesson_num,timestamp,transcript,elapsed_seconds
0,fast.ai 2022 - Part 1,2,00:00:00,"Hi everybody. Welcome to lesson two. Thanks for coming back… slight change of environment here,",0
1,fast.ai 2022 - Part 1,2,00:00:08,we had a bit of an “administrative issue” at our university — somebody booked our room — so I'm,8
2,fast.ai 2022 - Part 1,2,00:00:14,doing this from the study at home. so sorry about the lack of decorations behind me.,14
3,fast.ai 2022 - Part 1,2,00:00:25,I'm actually really really pumped about this lesson. It feels like going back to what things,25
4,fast.ai 2022 - Part 1,2,00:00:32,"were like in the very early days, because we're doing some really new, really cool stuff, which…",32


## Build our training data.  

This should be usable for both segmentation and summarization tasks

In [16]:
merged_df = topics_df[["course_title", "lesson_num", "topic", "start_seconds", "end_seconds"]].merge(
    transcripts_df, on=["course_title", "lesson_num"]
)
len(merged_df)

467129

Keep only the merged records where the transcript lies inbetween the start/end of the topic

In [17]:
merged_df = merged_df[
    (merged_df.elapsed_seconds >= merged_df.start_seconds) & (merged_df.elapsed_seconds < merged_df.end_seconds)
]

In [18]:
merged_df.head()

,course_title,lesson_num,topic,start_seconds,end_seconds,timestamp,transcript,elapsed_seconds
0,fast.ai 2022 - Part 1,2,Introduction,0,55,00:00:00,"Hi everybody. Welcome to lesson two. Thanks for coming back… slight change of environment here,",0
1,fast.ai 2022 - Part 1,2,Introduction,0,55,00:00:08,we had a bit of an “administrative issue” at our university — somebody booked our room — so I'm,8
2,fast.ai 2022 - Part 1,2,Introduction,0,55,00:00:14,doing this from the study at home. so sorry about the lack of decorations behind me.,14
3,fast.ai 2022 - Part 1,2,Introduction,0,55,00:00:25,I'm actually really really pumped about this lesson. It feels like going back to what things,25
4,fast.ai 2022 - Part 1,2,Introduction,0,55,00:00:32,"were like in the very early days, because we're doing some really new, really cool stuff, which…",32


For both segmentation and summarization tasks, we'll need to group the transcripts by course + lesson + topic

In [19]:
train_df = (
    merged_df[["course_title", "lesson_num", "topic", "transcript", "start_seconds"]]
    .groupby(by=["course_title", "lesson_num", "start_seconds", "topic"])
    .agg(list)
    .reset_index()
)

train_df.sort_values(by=["course_title", "lesson_num", "start_seconds"], inplace=True)

In [20]:
train_df.head()

,course_title,lesson_num,start_seconds,topic,transcript
0,C-Squared Podcast,1,0,Intro,"[[Music] welcome everybody to episode one of a, chess themed podcast with myself christian kirilla and i'm fighting on caruana so what's up, christian well not so much fabi uh it's first of all great um to finally start a, podcast the chess podcast i know that um there's a lot of podcasts out there but, i wanted to bring our own tune to the mix and i think uh yeah i'm, excited about that so that's uh the first thing how about yourself fabian well i'm back in the states after it's, been a while at your home it's good to be here it's my first time in uh visiting here and uh, yeah it's been a..."
1,C-Squared Podcast,1,137,Candidates 2018,"[camps look like in general yeah well you mentioned the 2018 cycle uh where we worked together we started with the, training before the candidates and for me it's interesting because i've i've played a lot of these, candidates tournaments and i'm always doing it a bit differently trying different things trying to improve it but sometimes it goes, less or more successfully you never know what will work out i think what we did in 2018 not just for the candidates but, also for the world championship because i qualified for that i think what we did then was extremely successful, um we we arran..."
2,C-Squared Podcast,1,464,Candidates training,"[going in the candidates like how was the experience yeah i think the preparation was pretty serious it, included a bunch of uh camps and preparation devoted to players as i assume i think everyone has the same, sort of general approach which is to think about their openings their strategy look at the opponents try to, get in shape make sure that you're not you know rusty or blundering things or hallucinating, variations uh but there's a lot of nerves and i i felt a lot of nerves before the tournament and i think possibly i, you know overworked over trained a bit because it was yeah it was..."
3,C-Squared Podcast,1,610,Playing for 2nd place,"[were you just like focused on grabbing first well i was only focused on first, but of course there were always these thoughts that well maybe second is enough but you can't play for second, like let's say once i had achieved plus three in the tournament and john was plus four and i tried to go and go into like full, like risk reverse mode which is still difficult to do but let's say i had gone that mode and, and achieved it and like finished second with like plus three and john got plus five uh, and then like magnus says well i'm going to play right then you also feel kind of stupid you k..."
4,C-Squared Podcast,1,916,Magnus' WC decision,"[know you can't uh you can't tell him you have to do something i i guess let me rephrase that, fair to let you guys play the tournament first and then tell you the decision, well i think he said it in a strange way which was that i'll play against alireza, which to me is strange because if you don't want to play world championship match i fully understand you know but did he say that did he actually name him, yeah that's kind of what he said um yeah he more he like he didn't say definitively like i won't play against, anyone but he was like i probably won't play unless it's frozen right an..."


## Build summarization training set

In [21]:
summarization_train_df = train_df.copy()

In [22]:
summarization_train_df["transcript"] = summarization_train_df["transcript"].apply(
    lambda v: " ".join([str(seq) for seq in v])
)

In [23]:
summarization_train_df.head()

,course_title,lesson_num,start_seconds,topic,transcript
0,C-Squared Podcast,1,0,Intro,[Music] welcome everybody to episode one of a chess themed podcast with myself christian kirilla and i'm fighting on caruana so what's up christian well not so much fabi uh it's first of all great um to finally start a podcast the chess podcast i know that um there's a lot of podcasts out there but i wanted to bring our own tune to the mix and i think uh yeah i'm excited about that so that's uh the first thing how about yourself fabian well i'm back in the states after it's been a while at your home it's good to be here it's my first time in uh visiting here and uh yeah it's been an intere...
1,C-Squared Podcast,1,137,Candidates 2018,camps look like in general yeah well you mentioned the 2018 cycle uh where we worked together we started with the training before the candidates and for me it's interesting because i've i've played a lot of these candidates tournaments and i'm always doing it a bit differently trying different things trying to improve it but sometimes it goes less or more successfully you never know what will work out i think what we did in 2018 not just for the candidates but also for the world championship because i qualified for that i think what we did then was extremely successful um we we arranged it...
2,C-Squared Podcast,1,464,Candidates training,going in the candidates like how was the experience yeah i think the preparation was pretty serious it included a bunch of uh camps and preparation devoted to players as i assume i think everyone has the same sort of general approach which is to think about their openings their strategy look at the opponents try to get in shape make sure that you're not you know rusty or blundering things or hallucinating variations uh but there's a lot of nerves and i i felt a lot of nerves before the tournament and i think possibly i you know overworked over trained a bit because it was yeah it was like ...
3,C-Squared Podcast,1,610,Playing for 2nd place,were you just like focused on grabbing first well i was only focused on first but of course there were always these thoughts that well maybe second is enough but you can't play for second like let's say once i had achieved plus three in the tournament and john was plus four and i tried to go and go into like full like risk reverse mode which is still difficult to do but let's say i had gone that mode and and achieved it and like finished second with like plus three and john got plus five uh and then like magnus says well i'm going to play right then you also feel kind of stupid you know li...
4,C-Squared Podcast,1,916,Magnus' WC decision,know you can't uh you can't tell him you have to do something i i guess let me rephrase that fair to let you guys play the tournament first and then tell you the decision well i think he said it in a strange way which was that i'll play against alireza which to me is strange because if you don't want to play world championship match i fully understand you know but did he say that did he actually name him yeah that's kind of what he said um yeah he more he like he didn't say definitively like i won't play against anyone but he was like i probably won't play unless it's frozen right and yeah...


In [24]:
summarization_train_df.iloc[0].transcript

"[Music] welcome everybody to episode one of a chess themed podcast with myself christian kirilla and i'm fighting on caruana so what's up christian well not so much fabi uh it's first of all great um to finally start a podcast the chess podcast i know that um there's a lot of podcasts out there but i wanted to bring our own tune to the mix and i think uh yeah i'm excited about that so that's uh the first thing how about yourself fabian well i'm back in the states after it's been a while at your home it's good to be here it's my first time in uh visiting here and uh yeah it's been an interesting few months played a lot of chess which is pretty cool but also a bit difficult at times my home uh here we are not going to mention the location because those uh crazy fans who knows maybe they will uh track me down so you're back in the states you've been away for uh quite a few months actually you were in europe for a very long time when's the last time you were in the states well i played th

In [25]:
summarization_train_df.shape

(597, 5)

## Blurr learner for training summarization model

In [24]:
print(f"Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}")

Using GPU #0: Tesla V100-SXM2-16GB


In [25]:
learn = BlearnerForSummarization.from_data(
    summarization_train_df,
    "sshleifer/distilbart-cnn-6-6",
    text_attr="transcript",
    summary_attr="topic",
    max_length=256,
    max_target_length=130,
    dblock_splitter=RandomSplitter(),
    dl_kwargs={"bs": 32},
).to_fp16()

/home/team_007/mambaforge/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [30]:
learn.fit_one_cycle(10, lr_max=4e-5, cbs=[BlearnerForSummarization.get_metrics_cb()])

epoch,train_loss,valid_loss,rouge1,rouge2,rougeL,rougeLsum,bertscore_precision,bertscore_recall,bertscore_f1,rouge1,rouge2,rougeL,rougeLsum,bertscore_precision,bertscore_recall,bertscore_f1,time
0,3.996448,3.738628,0.070186,0.012283,0.058038,0.064639,0.808916,0.867189,0.836888,0.063692,0.012697,0.055451,0.061330,0.809005,0.865657,0.836259,00:18
1,3.780062,3.469175,0.070186,0.012283,0.058038,0.064639,0.808916,0.867189,0.836888,0.068255,0.015796,0.058577,0.063986,0.810859,0.868150,0.838391,00:14
2,3.505078,3.326282,0.070186,0.012283,0.058038,0.064639,0.808916,0.867189,0.836888,0.078303,0.021914,0.070290,0.073146,0.809613,0.871336,0.839143,00:13
3,3.194408,3.310852,0.070186,0.012283,0.058038,0.064639,0.808916,0.867189,0.836888,0.073724,0.022267,0.064284,0.067749,0.805505,0.869810,0.836239,00:13
4,2.896657,3.416538,0.070186,0.012283,0.058038,0.064639,0.808916,0.867189,0.836888,0.082064,0.023593,0.074779,0.078019,0.808542,0.871605,0.838720,00:13
5,2.616342,3.528751,0.070186,0.012283,0.058038,0.064639,0.808916,0.867189,0.836888,0.072377,0.018873,0.065467,0.067861,0.798134,0.867135,0.830999,00:13
6,2.361720,3.633969,0.070186,0.012283,0.058038,0.064639,0.808916,0.867189,0.836888,0.079125,0.023314,0.071311,0.075502,0.802681,0.868481,0.834081,00:13
7,2.149974,3.707175,0.070186,0.012283,0.058038,0.064639,0.808916,0.867189,0.836888,0.076346,0.019703,0.069976,0.071863,0.800788,0.867932,0.832786,00:13
8,1.971459,3.737713,0.070186,0.012283,0.058038,0.064639,0.808916,0.867189,0.836888,0.079448,0.017940,0.072168,0.073141,0.800576,0.867136,0.832307,00:13
9,1.831894,3.741242,0.070186,0.012283,0.058038,0.064639,0.808916,0.867189,0.836888,0.074097,0.016569,0.067867,0.069131,0.800944,0.866961,0.832428,00:13


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/home/team_007/mambaforge/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/team_007/mambaforge/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/team_007/mambaforge/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/team_007/mambaforge/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


...

/home/team_007/mambaforge/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/team_007/mambaforge/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/team_007/mambaforge/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


## Predictions and taking look at results

In [32]:
learn.show_results(learner=learn)

In [33]:
test_article = """hey everybody welcome back this week we're going to talk about something a little bit different than we do most weeks most weeks we talk about specific
technical aspects of building machine learning powered products but this week we're going to focus on some of the
organizational things that you need to do in order to work together on ml-powered products as part of an
interdisciplinary team so the the reality of building ml Power Products is that building any product well is really
difficult you have to figure out how to hire grade people you need to be able to manage those people and get the best out
of them you need to make sure that your team is all working together towards a shared goal you need to make good
long-term technical choices manage technical debt over time you need to make sure that you're managing
expectations not just of your own team but also of leadership of your organization and you need to be able to make sure
that you're working well within the confines of the requirements of the rest of the org that you're understanding
those requirements well and communicating back to your progress to the rest of the organization against those requirements
but machine learning adds even more additional complexity to this machine learning Talent tends to be very scarce
and expensive to attract machine learning teams are not just a
single role but today they tend to be pretty interdisciplinary which makes managing them an even bigger challenge
machine learning projects often have unclear timelines and there's a high
degree of uncertainty to those timelines machine learning itself is moving super fast and machine learning as we've
covered before you can think of as like the high interest credit card of technical debt so keeping up with making
good long-term decisions and not incurring too much technical debt is especially difficult in ml unlike
traditional software ml is so new that in most organizations leadership tends not to be that well educated in it they
might not understand some of the core differences between ML and other technology that you're working with machine learning products tend to fail
in ways that are really hard for Lay people to understand and so that makes it very difficult to help the rest of
the stakeholders in your organization understand what they could really expect from the technology that you're building
and what is realistic for us to achieve so throughout the rest rest of this lecture we're going to kind of touch on
some of these themes and cover different aspects of this problem of working together to build ml Power Products as
an organization so here are the pieces that we're going to cover we're going to talk about different roles that are involved in building ml products we're
going to talk about some of the unique aspects involved in hiring ml Talent
we're going to talk about organization of teams and how the ml team tends to fit into the rest of the org and some of
the pros and cons of different ways of setting that up we'll talk about managing ml teams and
ml product management and then lastly we'll talk about some of the design considerations for how to design a
product that is well suited to having a good ml model that backs it so let's dive in and talk about rules the most
common ml rules that you might hear of are things like ml product manager ml
"""

In [34]:
learn.predict(test_article, num_return_sequences=3)

/home/team_007/mambaforge/lib/python3.10/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 142 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'summary_texts': [' What To Hire (Organization/Supplier/Hire/Hole) The Top 5 ML Teams (Organs, or Not Top 10 ML Teams) Back to Mail Online home. Back to the page you came from.com/Top 5 ML Partners (Note: YOLO)',
   ' What To Hire (Organization/Supplier/Hire/Hole) The Top 5 ML Teams (Organs, or Not Top 10 ML Teams) Back to Mail Online home. Back to the page you came from.com/Top 5 ML Partners (UPS)',
   ' What To Hire (Organization/Supplier/Hire/Hole) The Top 5 ML Teams (Organs, or Not Top 10 ML Teams) Back to Mail Online home. Back to the page you came from.com/Locked/Top 5']}]